# Goals
- [X] Comparison of country voting alignment with China, over time for all countries, absolute number of votes cast (bubble graph? Geographic heat map?)
- [ ] Comparison of financial assistance received from China, over time for all countries (bubble graph? Geographic heat map?)
      
- [ ] Overall voting alignment with China per vote type/topic, over time for all countries (bubble graph?)

- [ ] Voting alignment with China and financial assistance, over time, per country (2-track bar chart?)
- [ ] Country voting alignment with China pre- and post-vaccine donations, per country (2-track bar chart?)

- [ ] Comparison of country change in voting alignment with financial assistance received (smallest -> largest positive change in voting alignment / amount of financial assistance)
- [ ] Comparison of country change in voting alignment with donated vaccines received (smallest -> largest positive change in voting alignment / vaccines donated)
- [ ] Comparison of change in financial flows to country compared to voting alignment, over time (smallest -> largest increase in flows / smallest -> largest positive change in voting alignment)
      
- [ ] Comparative country Group recipients of Chinese financial aid, over time (donut chart? Bump area chart?)
- [ ] Comparative voting alignment of country Group with China, over time (donut chart? Bump area chart?)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import geopandas as gpd
from bokeh.io import output_notebook, show
from bokeh.models import GeoJSONDataSource, Slider, CustomJS, LinearColorMapper, Range1d
from bokeh.models import HoverTool
from bokeh.plotting import figure
from bokeh.layouts import column, row
import json
from ipywidgets import interact
import ipywidgets as widgets
from bokeh.models import ColumnDataSource
output_notebook()

In [ ]:
# Load world geometry data
world = gpd.read_file('data/ne_110m_admin_0_countries.shp')

AFR = ['Algeria','Angola','Benin','Botswana,Burkina Faso','Burundi','Cameroon','Congo','Côte d’Ivoire','Democratic Republic of the Congo','Djibouti','Egypt','Eritrea','Ethiopia','Gabon','The Gambia','Ghana','Kenya','Libya','Madagascar','Malawi','Mali','Mauritania','Mauritius','Morocco','Namibia','Nigeria','Rwanda','Senegal','Sierra Leone','South Africa','Somalia','Sudan','Togo','Tunisia','Uganda','Zambia']
APAC = ['Afghanistan','Azerbaijan','Bahrain','Bangladesh','China','Fiji','India','Indonesia','Iraq','Japan','Jordan','Kazakhstan','Kuwait','Kyrgyzstan','Malaysia','Maldives','Marshall Islands','Mongolia','Nepal','Pakistan','Philippines','Qatar','Republic of Korea','Saudi Arabia','Sri Lanka','Thailand','United Arab Emirates','Uzbekistan','Viet Nam']
GRULAC=['Argentina','Bahamas','Bolivia (Plurinational State of)','Brazil','Chile','Costa Rica','Cuba','Ecuador','El Salvador','Guatemala','Honduras','Mexico','Nicaragua','Panama','Paraguay','Peru','Uruguay','Venezuela (Bolivarian Republic of)']
WEOG = ['Australia','Austria','Belgium','Canada','Denmark','Finland','France','Germany','Iceland','Ireland','Italy','Luxembourg','Netherlands','Norway','Portugal','Spain','Switzerland','United Kingdom of Great Britain and Northern Ireland','United States of America']
EG = ['Albania','Armenia','Bosnia and Herzegovina','Bulgaria','Czechia','Croatia','Estonia','Georgia','Hungary','Latvia','Lithuania','Montenegro','Poland','Republic of Moldova','Republic of North Macedonia','Romania','Russian Federation','Slovakia','Slovenia','Ukraine']

# Function to assign color based on country group
def assign_color(country):
    if country in AFR:
        return 'blue'
    elif country in APAC:
        return 'green'
    elif country in GRULAC:
        return 'red'
    elif country in WEOG:
        return 'yellow'
    elif country in EG:
        return 'orange'
    else:
        return 'lightgray'

## Comparison of country voting alignment with China and Financial Assistance
over time for all countries, absolute number of votes cast

In [ ]:
import colorcet

# Load your data
df = pd.read_csv("data/oda.csv")

# Create a new column 'vote_nr' based on the values in the 'Vote' column
china_votes = df[df["Country"] == "China"][
    ["Session number", "Text title", "Vote"]
].rename(columns={"Vote": "China Vote"})
df = df.merge(
    china_votes,
    on=["Session number", "Text title"],
    suffixes=("", "_china"),
    how="left",
)
# Create a new column 'mapped_vote' that maps three values to -1, 0, and 1
vote_mapping = {"Against": -1, "Abstaining": 0, "In Favour": 1}
df["mapped_vote"] = df["Vote"].map(vote_mapping)
df["mapped_china_vote"] = df["China Vote"].map(vote_mapping)

def calculate_alignment(row):
    if row["Vote"] == row["China Vote"]:
        return 1
    else:
        return -1

df["alignment_score"] = df.apply(calculate_alignment, axis=1)
df_aggregated = (
    df.groupby(["Year", "Country"])
    .agg(
        {
            "alignment_score": "sum",
            "Text title": "count",
            "ODI_MOFCOM": "first",
            "BU_ODA": "first",
            "AEI_FDI": "first",
            "FT_ODI": "first",
        }
    )
    .reset_index()
)
df_aggregated.rename(columns={"Text title": "Number of votes"}, inplace=True)
df_aggregated["alignment"] = (
    df_aggregated["alignment_score"] / df_aggregated["Number of votes"]
)
df_aggregated["alignment_percentage"] = (
    df_aggregated["alignment_score"] / df_aggregated["Number of votes"]
) * 100
years = pd.DataFrame({"Year": df["Year"].unique()})
topics = df["Topic"].dropna().unique()

# Load world geometry data (you might need to adjust the file path)
countries = pd.DataFrame({"Country": world["ADMIN"].unique()})

cya = years.merge(countries, how="cross")
cya = cya.merge(df_aggregated, on=["Year", "Country"], how="left")
# Merge your aggregated data with the geopandas DataFrame
merged = world.merge(cya, left_on="ADMIN", right_on="Country", how="left")
merged = merged[
    [
        "featurecla",
        "CONTINENT",
        "REGION_UN",
        "ADMIN",
        "SUBREGION",
        "REGION_WB",
        "MIN_ZOOM",
        "MIN_LABEL",
        "MAX_LABEL",
        "LABEL_X",
        "LABEL_Y",
        "geometry",
        "Year",
        "Country",
        "alignment_score",
        "Number of votes",
        "alignment",
        "alignment_percentage",
        "FT_ODI",
        "AEI_FDI",
        "BU_ODA",
        "ODI_MOFCOM",
    ]
]
merged_initial = merged[(merged["Year"] == 2006) | merged["Year"].isna()]
# Convert to GeoJSON
geojson = json.dumps(merged_initial.__geo_interface__)
geojson_original = json.dumps(merged.__geo_interface__)
# Input GeoJSON source that contains features for plotting
geosource = GeoJSONDataSource(geojson=geojson)

# Determine the bounds for your map
x_range = Range1d(start=-180, end=180)
y_range = Range1d(start=-90, end=90)


def render_layout(fa_source):
    tools = "wheel_zoom,pan,reset"
    va_map = figure(
        title="Voting Alignment with China",
        height=600,
        width=950,
        x_range=x_range,
        y_range=y_range,
        tools=tools,
    )

    # Define color mapper
    color_mapper = LinearColorMapper(
        palette=colorcet.b_diverging_gwr_55_95_c38, low=100, high=-100
    )

    # Add patch renderer to figure
    va_map.patches(
        "xs",
        "ys",
        source=geosource,
        fill_color={"field": "alignment_percentage", "transform": color_mapper},
        line_color="black",
        line_width=0.5,
    )
    tooltips = [
        ("Country", "@Country"),
        ("Alignment Percent", "@alignment"),
    ]

    va_map.add_tools(HoverTool(tooltips=tooltips))

    slider = Slider(
        start=df["Year"].min(),
        end=df["Year"].max(),
        value=df["Year"].min(),
        step=1,
        title="Year",
    )
    callback = CustomJS(
        args=dict(source=geosource, available=geojson_original, slider=slider),
        code="""
        const year = slider.value;
        const geojson = (JSON.parse(source.geojson))
        const features = JSON.parse(available).features.filter(feature => feature.properties.Year === year);
        const new_geojson = {type:geojson.type, features:features}

        source.geojson = JSON.stringify(new_geojson);
        source.change.emit();
    """,
    )
    slider.js_on_change("value", callback)
    fa_color_mapper = LinearColorMapper(
        palette=colorcet.b_diverging_gwr_55_95_c38,
        low=0,
        high=df_aggregated[["FT_ODI", "AEI_FDI", "BU_ODA", "ODI_MOFCOM"]]
        .max(axis=1)
        .max(),
    )
    fa_map = figure(
        title="Financial Assistance from China",
        height=600,
        width=950,
        x_range=x_range,
        y_range=y_range,
        tools=tools,
    )
    fa_map.patches(
        "xs",
        "ys",
        source=geosource,
        fill_color={"field": fa_source, "transform": fa_color_mapper},
        line_color="black",
        line_width=0.5,
    )
    tooltips = [
        ("Country", "@Country"),
        ("Alignment Percent", "@alignment"),
        ("Financial Assistance", f"@{fa_source}"),
        ("Financial Assistance type", fa_source),
    ]
    fa_map.add_tools(HoverTool(tooltips=tooltips))
    layout = column(slider, row(va_map, fa_map))
    show(layout)


interact(
    render_layout,
    fa_source=widgets.Dropdown(
        options=["FT_ODI", "AEI_FDI", "BU_ODA", "ODI_MOFCOM"], description="FA Source:", value="FT_ODI"
    ),
)


## Comparison of country voting alignment with China
over time for all countries, absolute number of votes cast

In [ ]:
# Load your data
df = pd.read_csv("data/oda.csv")

# Create a new column 'vote_nr' based on the values in the 'Vote' column

china_votes = df[df["Country"] == "China"][
    ["Session number", "Text title", "Vote"]
].rename(columns={"Vote": "China Vote"})
df = df.merge(
    china_votes,
    on=["Session number", "Text title"],
    suffixes=("", "_china"),
    how="left",
)

# Create a new column 'mapped_vote' that maps three values to -1, 0, and 1
vote_mapping = {"Against": -1, "Abstaining": 0, "In Favour": 1}
df["mapped_vote"] = df["Vote"].map(vote_mapping)
df["mapped_china_vote"] = df["China Vote"].map(vote_mapping)


def calculate_alignment(row):
    if row["Vote"] == row["China Vote"]:
        return 1
    else:
        return -1


df["alignment_score"] = df.apply(calculate_alignment, axis=1)
df_aggregated = (
    df.groupby(["Year", "Country"])
    .agg(
        {
            "alignment_score": "sum",
            "Text title": "count",
            "ODI_MOFCOM": "first",
            "BU_ODA": "first",
            "AEI_FDI": "first",
            "FT_ODI": "first",
        }
    )
    .reset_index()
)
df_aggregated.rename(columns={"Text title": "Number of votes"}, inplace=True)
df_aggregated["alignment"] = (
    df_aggregated["alignment_score"] / df_aggregated["Number of votes"]
)
df_aggregated["alignment_percentage"] = (
    df_aggregated["alignment_score"] / df_aggregated["Number of votes"]
) * 100

# Load world geometry data (you might need to adjust the file path)
years = pd.DataFrame({"Year": df["Year"].unique()})
countries = pd.DataFrame({"Country": world["ADMIN"].unique()})

cya = years.merge(countries, how="cross")
cya = cya.merge(df_aggregated, on=["Year", "Country"], how="left")
# Merge your aggregated data with the geopandas DataFrame
cya = cya[
    [
        "Year",
        "Country",
        "ODI_MOFCOM",
        "BU_ODA",
        "AEI_FDI",
        "FT_ODI",
        "alignment_score",
        "Number of votes",
        "alignment",
        "alignment_percentage",
    ]
]
cya = cya.astype({'Year': 'string'})
cya.dropna()
from ipywidgets import interact
import ipywidgets as widgets
from bokeh.layouts import gridplot
from bokeh.plotting import figure
import matplotlib.pyplot as plt
import numpy as np
from bokeh.models import LinearAxis, Range1d

list_plots = []


def plot(fa_type, nr_plots):
    for country in cya["Country"].unique().tolist():
        temp = cya[cya["Country"] == country]
        if temp[fa_type].any():
            plot = figure(width=450, height=350, title=country)
            plot.y_range = Range1d(start=-100, end=+100)
            plot.extra_y_ranges = {"FA": Range1d(start=0, end=temp[fa_type].max())}
            plot.add_layout(LinearAxis(y_range_name="FA", axis_label="Financial Assistance"), "right")
            plot.circle(x=temp["Year"].unique().tolist(), y=temp["alignment_percentage"].tolist())
            plot.line(x=temp["Year"].unique().tolist(), y=temp["alignment_percentage"].tolist(), legend_label="Alignment percentage")
            plot.square(x=temp["Year"].unique().tolist(), y=temp[fa_type], y_range_name="FA", fill_color="red")
            plot.line(x=temp["Year"].unique().tolist(), y=temp[fa_type], y_range_name="FA", line_color="red", legend_label="Financial Assitance")
            list_plots.append(plot)
    x = nr_plots
    grid = [list_plots[i : i + x] for i in range(0, len(list_plots), x)]
    p = gridplot(grid)
    show(p)


interact(
    plot,
    fa_type=widgets.Dropdown(
        options=["FT_ODI", "AEI_FDI", "BU_ODA", "ODI_MOFCOM"], description="FA Type:"
    ),
    nr_plots=widgets.BoundedIntText(
        value=4,
        min=2,
        max=10,
        step=1,
        description="Plots (2 to 10):",
        disabled=False,
    ),
)
plot("FT_ODI", 4)
cya.head()

In [ ]:
# Group by 'Year' and 'Country' and calculate the sum of 'Chinainfav'
df_summary = df.groupby(['Year', 'Country']).agg({
    'In favour': 'sum',
    'Against': 'sum',
    'Chinainfav': lambda x: (x == 1).sum(),
    'Chinagainst': lambda x: (x == 1).sum()
}).reset_index()
df_summary['Total'] = df_summary['Chinainfav'] + df_summary['Chinagainst']
# Display the table
print(df_summary)

In [ ]:
import pandas as pd
import geopandas as gpd
from itertools import product
from bokeh.models import ColumnDataSource, CustomJS, LogColorMapper, Slider
from bokeh.plotting import figure, show, output_file
from bokeh.layouts import column

# Read the data
data = pd.read_csv("data/oda.csv")

# Calculate total votes in favor per country per year
total_votes_favor = data.groupby(['Country', 'Year'])['In favour'].sum().reset_index()

# Normalize data - ensure each country has an entry for each year
all_countries = total_votes_favor['Country'].unique()
all_years = total_votes_favor['Year'].unique()

all_combinations = pd.DataFrame(product(all_countries, all_years), columns=['Country', 'Year'])
total_votes_favor = pd.merge(all_combinations, total_votes_favor, on=['Country', 'Year'], how='left')
total_votes_favor['In favour'].fillna(0, inplace=True)

# Load world geometry data
world = gpd.read_file('data/ne_110m_admin_0_countries.shp')

# Reproject to a projected CRS (e.g., World Mercator)
world = world.to_crs(epsg=3395)

# Calculate the centroid of each country's geometry
world['centroid'] = world['geometry'].centroid

# Convert back to geographic CRS for latitude and longitude
world = world.to_crs(epsg=4326)

# Extract the longitude and latitude of the centroid
world['lon'] = world['centroid'].x
world['lat'] = world['centroid'].y

# Create a dictionary with country names as keys and coordinates as values
country_coords = world.set_index('ADMIN')['centroid'].apply(lambda x: {'lon': x.x, 'lat': x.y}).to_dict()

# Prepare data dictionary for ColumnDataSource
data_dict = {'x': [], 'y': [], 'name': [], 'votes': []}
for year in all_years:
    year_data = total_votes_favor[total_votes_favor['Year'] == year]
    for country in all_countries:
        if country in country_coords:
            data_dict['x'].append(country_coords[country]['lon'])
            data_dict['y'].append(country_coords[country]['lat'])
            data_dict['name'].append(country)
            data_dict['votes'].append(year_data[year_data['Country'] == country]['In favour'].iloc[0])
            data_dict[str(year)] = data_dict['votes'].copy()

# Create a ColumnDataSource
source = ColumnDataSource(data_dict)

print(source)

# Define color mapper
custom_colors = ['#f2f2f2', '#fee5d9', '#fcbba1', '#fc9272', '#fb6a4a', '#de2d26']
color_mapper = LogColorMapper(palette=custom_colors)

# Create figure
TOOLS = "pan,wheel_zoom,reset,hover,save"
p = figure(title="Total Votes in Favor per Country per Year", tools=TOOLS, x_axis_location=None, y_axis_location=None)
p.grid.grid_line_color = None

# Plot the countries
p.patches('x', 'y', source=source, fill_color={'field': 'votes', 'transform': color_mapper}, fill_alpha=0.8, line_color="black", line_width=0.5)

# Slider
slider = Slider(start=min(all_years), end=max(all_years), value=min(all_years), step=1, title="Year")

# JavaScript callback for the slider
callback = CustomJS(args=dict(source=source, slider=slider), code="""
var data = source.data;
var year = slider.value.toString();
data['votes'] = data[year];
source.change.emit();
""")

slider.js_on_change('value', callback)

# Show plot
output_notebook()
show(column(slider, p))

## Comparison of financial assistance received from Chinaover time for all countries